In [2]:
import cv2
import numpy as np
import pandas as pd

# Constants
video_file = '29_video.mkv'  # Replace with your video file path

# Open the video file
cap = cv2.VideoCapture(video_file)
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second of the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frames in the video

# Calculate the frame range
duration_seconds = 15 * 60  # 15 minutes in seconds
frames_to_process = duration_seconds * fps  # Frames to process for 15 minutes
end_frame = total_frames - (60 * fps)  # Exclude the last 60 seconds
start_frame = max(0, end_frame - frames_to_process)  # Start frame for the last 15 minutes excluding last 60 seconds

# Initialize parameters for optical flow calculation
prev_gray = None
optical_flow_values = []

# Set the starting frame
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Loop over frames in the specified range
for i in range(start_frame, end_frame):
    ret, frame = cap.read()
    if not ret:
        break

    # Crop the central 80% of the frame
    height, width = frame.shape[:2]
    crop_width = int(width * 0.8)
    crop_height = int(height * 0.8)
    x_start = (width - crop_width) // 2
    y_start = (height - crop_height) // 2
    frame_cropped = frame[y_start:y_start+crop_height, x_start:x_start+crop_width]

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow if there's a previous frame
    if prev_gray is not None:
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        avg_magnitude = np.mean(mag)
        optical_flow_values.append(avg_magnitude)

    # Set the current frame to previous
    prev_gray = gray

# Downsample the optical flow values to 1Hz (1 value per second)
downsampled_data = []
for i in range(0, len(optical_flow_values)):
    sample_mean = np.mean(optical_flow_values[i:i+fps])
    sample_median = np.median(optical_flow_values[i:i+fps])
    downsampled_data.append({'mean': sample_mean, 'median': sample_median})

# Create a DataFrame
df = pd.DataFrame(downsampled_data)

# Output the DataFrame
print(df)

# Release the video capture
cap.release()


         mean    median
0    0.178822  0.154815
1    0.100140  0.084262
2    0.104509  0.100199
3    0.121259  0.119429
4    0.089803  0.093231
..        ...       ...
893  3.825157  4.495791
894  3.305480  2.956156
895  5.185573  6.372876
896  5.485275  5.246538
897  3.169283  3.697870

[898 rows x 2 columns]


In [18]:
# Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates

In [5]:
df.describe()

,mean,median
count,898.000000,898.000000
mean,3.418124,3.327080
std,1.446643,1.751254
min,0.073743,0.010429
25%,2.573835,2.255070
50%,3.447178,3.289779
75%,4.304537,4.301511
max,8.849064,10.093546


In [7]:
df = df.head(840)

In [9]:
df1

,fms
0,1
1,1
2,1
3,1
4,1
...,...
835,2
836,2
837,2
838,2


In [6]:
df1 = pd.read_csv("fms.csv")

In [10]:
df = pd.concat([df1, df], axis=1)

In [11]:
df

,fms,mean,median
0,1,0.178822,0.154815
1,1,0.100140,0.084262
2,1,0.104509,0.100199
3,1,0.121259,0.119429
4,1,0.089803,0.093231
...,...,...,...
835,2,2.580304,3.019026
836,2,2.793762,1.174016
837,2,4.629346,4.088746
838,2,3.701176,3.504064


In [12]:
def prepare_time_series_data(data, time_step=1, output_dim=1, drop_na=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  for i in range(time_step, 0, -1):
      cols.append(df.shift(i))
      names += [('X%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
  for i in range(0, output_dim):
      cols.append(df.shift(-i))
      if i == 0:
          names += [('Y%d(t)' % (j + 1)) for j in range(n_vars)]
      else:
          names += [('Y%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
  aggregated_data = pd.concat(cols, axis=1)
  aggregated_data.columns = names
  if drop_na:
      aggregated_data.dropna(inplace=True)
  return aggregated_data

In [13]:
forecast_horizon = 1
time_lag = 30

# Convert the data to time-series
train_hr_data_ml = prepare_time_series_data(data=df, time_step=time_lag, output_dim=forecast_horizon)
test_hr_data_ml =  prepare_time_series_data(data=df, time_step=time_lag, output_dim=forecast_horizon)
train_hr_data_ml.head(2)

,X1(t-30),X2(t-30),X3(t-30),X1(t-29),X2(t-29),X3(t-29),X1(t-28),X2(t-28),X3(t-28),X1(t-27),...,X3(t-3),X1(t-2),X2(t-2),X3(t-2),X1(t-1),X2(t-1),X3(t-1),Y1(t),Y2(t),Y3(t)
30,1.0,0.178822,0.154815,1.0,0.100140,0.084262,1.0,0.104509,0.100199,1.0,...,0.117719,1.0,0.173790,0.136031,1.0,0.387818,0.255108,1,0.381167,0.351902
31,1.0,0.100140,0.084262,1.0,0.104509,0.100199,1.0,0.121259,0.119429,1.0,...,0.136031,1.0,0.387818,0.255108,1.0,0.381167,0.351902,1,1.192985,1.456125


In [14]:
train_hr_data_ml = train_hr_data_ml[train_hr_data_ml.columns[~train_hr_data_ml.columns.to_series().str.contains(pat='X1\(')]]
train_hr_data_ml = train_hr_data_ml[train_hr_data_ml.columns[~train_hr_data_ml.columns.to_series().str.contains(pat='Y2\(')]]

test_hr_data_ml = test_hr_data_ml[test_hr_data_ml.columns[~test_hr_data_ml.columns.to_series().str.contains(pat='X1\(')]]
test_hr_data_ml = test_hr_data_ml[test_hr_data_ml.columns[~test_hr_data_ml.columns.to_series().str.contains(pat='Y2\(')]]


train_hr_data_ml.head(2)

,X2(t-30),X3(t-30),X2(t-29),X3(t-29),X2(t-28),X3(t-28),X2(t-27),X3(t-27),X2(t-26),X3(t-26),...,X2(t-4),X3(t-4),X2(t-3),X3(t-3),X2(t-2),X3(t-2),X2(t-1),X3(t-1),Y1(t),Y3(t)
30,0.178822,0.154815,0.100140,0.084262,0.104509,0.100199,0.121259,0.119429,0.089803,0.093231,...,0.121841,0.110027,0.112853,0.117719,0.173790,0.136031,0.387818,0.255108,1,0.351902
31,0.100140,0.084262,0.104509,0.100199,0.121259,0.119429,0.089803,0.093231,0.109756,0.107062,...,0.112853,0.117719,0.173790,0.136031,0.387818,0.255108,0.381167,0.351902,1,1.456125


In [26]:
def get_x_y_data(data, time_step, number_of_features):
  values = data.values
  number_observation = time_step * number_of_features
  X, Y = values[:, :number_observation], values[:, -2]

  scaler = StandardScaler()
  X = scaler.fit_transform(X)
  X = X.reshape((X.shape[0], time_step, number_of_features))  # Reshape as (sample, time_step, features)

  return X, Y

In [27]:
train_HR_X, train_HR_Y = get_x_y_data(data=train_hr_data_ml, time_step=30,
                                      number_of_features=1) # Note HR data has only one featue which is the HR value


test_HR_X, test_HR_Y = get_x_y_data(data=test_hr_data_ml, time_step=30,
                                      number_of_features=1)

print("Reshape as (sample, time_step, features)\n")
print("Train HR data shape: ", train_HR_X.shape)
print("Train HR Target shape: ", train_HR_Y.shape)

print("Test HR data shape: ", test_HR_X.shape)
print("Test HR Target shape: ", test_HR_Y.shape)

Reshape as (sample, time_step, features)

Train HR data shape:  (810, 30, 1)
Train HR Target shape:  (810,)
Test HR data shape:  (810, 30, 1)
Test HR Target shape:  (810,)


In [20]:
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout, concatenate
from tensorflow.keras.models import Model

In [21]:

def get_lstm(input_shape):
  if not input_shape:
      print("Error No input shape defined")
      return
  input_layer = Input(shape=input_shape)
  X = LSTM(60, return_sequences=True)(input_layer)
  X = LSTM(120, recurrent_dropout=0.2)(X)
  X = Dense(256, activation='relu')(X)
  X = Dropout(0.2)(X)
  output = Dense(64, activation='relu')(X)
  return input_layer, output

def get_regression_model(input_layers, output_layers, merge=True):
  if merge:
      merge_layer = concatenate(output_layers)
  else:
      merge_layer = output_layers[0]
  final = Dense(1)(merge_layer)
  model = Model(input_layers, outputs=final)
  return model

In [22]:
input_layer1, output1 = get_lstm(input_shape=(30, 1))

In [23]:
regression_model = get_regression_model(input_layers=[input_layer1],
                                                    output_layers=[output1], merge=True)
regression_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 30, 60)            14880     
                                                                 
 lstm_1 (LSTM)               (None, 120)               86880     
                                                                 
 dense (Dense)               (None, 256)               30976     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 concatenate (Concatenate)   (None, 64)                0     

In [24]:
regression_model.compile(loss='mse', optimizer='adam', metrics=['mae', 'msle'])

In [28]:
train_HR_Y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [30]:
train_HR_X =np.asarray(train_HR_X[1:]).astype(int) 
train_HR_Y = np.asarray(train_HR_Y).astype(int)

history = regression_model.fit(x=[train_HR_X], y=train_HR_Y, epochs=20,
                                   batch_size=128, validation_split=0.2, verbose=1, shuffle=False)

Epoch 1/20


6/6 [==============================] - 5s 202ms/step - loss: 0.9466 - mae: 0.9724 - msle: 0.4404 - val_loss: 1.7815 - val_mae: 1.2445 - val_msle: 0.5689
Epoch 2/20
6/6 [==============================] - 0s 49ms/step - loss: 0.5481 - mae: 0.7292 - msle: 0.2194 - val_loss: 0.5206 - val_mae: 0.5644 - val_msle: 0.0982
Epoch 3/20
6/6 [==============================] - 0s 46ms/step - loss: 0.2722 - mae: 0.2548 - msle: 0.0372 - val_loss: 0.3715 - val_mae: 0.4461 - val_msle: 0.0634
Epoch 4/20
6/6 [==============================] - 0s 45ms/step - loss: 0.0423 - mae: 0.1674 - msle: 0.0116 - val_loss: 0.4370 - val_mae: 0.4644 - val_msle: 0.0771
Epoch 5/20
6/6 [==============================] - 0s 41ms/step - loss: 0.0384 - mae: 0.1524 - msle: 0.0110 - val_loss: 0.3493 - val_mae: 0.3975 - val_msle: 0.0573
Epoch 6/20
6/6 [==============================] - 0s 42ms/step - loss: 0.0113 - mae: 0.0829 - msle: 0.0029 - val_loss: 0.3829 - val_mae: 0.3912 - val_msle: 0.0637
Epoch 7/20
6/6 [===

In [3]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from skimage.color import rgb2gray
import os



In [1]:
def extract_optical_flow(video_path, start_frame, end_frame):
    cap = cv2.VideoCapture(video_path)
    prev_gray = None
    optical_flow_values = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    for i in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break

        height, width = frame.shape[:2]
        crop_width = int(width * 0.8)
        crop_height = int(height * 0.8)
        x_start = (width - crop_width) // 2
        y_start = (height - crop_height) // 2
        frame_cropped = frame[y_start:y_start+crop_height, x_start:x_start+crop_width]

        gray = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            avg_magnitude = np.mean(mag)
            optical_flow_values.append(avg_magnitude)

        prev_gray = gray

    cap.release()
    
    downsampled_data = []
    for i in range(0, len(optical_flow_values)):
        sample_mean = np.mean(optical_flow_values[i:i+fps])
        sample_median = np.median(optical_flow_values[i:i+fps])
        downsampled_data.append({'mean_optical_flow': sample_mean, 'median_optical_flow': sample_median})

    return pd.DataFrame(downsampled_data)

def extract_hog_features(video_path, start_frame, end_frame):
    cap = cv2.VideoCapture(video_path)
    hog_features = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    for i in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = rgb2gray(frame)
        hog_feature, _ = hog(gray_frame, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
        hog_features.append(hog_feature)

    cap.release()

    downsampled_hog = []
    for i in range(0, len(hog_features)):
        sample_hog = np.mean(hog_features[i:i+fps], axis=0)
        downsampled_hog.append(sample_hog)

    return downsampled_hog

def extract_color_histogram(video_path, start_frame, end_frame):
    cap = cv2.VideoCapture(video_path)
    histograms = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    for i in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break

        hist_b = cv2.calcHist([frame], [0], None, [256], [0, 256])
        hist_g = cv2.calcHist([frame], [1], None, [256], [0, 256])
        hist_r = cv2.calcHist([frame], [2], None, [256], [0, 256])
        
        histograms.append(np.concatenate((hist_b, hist_g, hist_r)).flatten())

    cap.release()

    downsampled_hist = []
    for i in range(0, len(histograms)):
        sample_hist = np.mean(histograms[i:i+fps], axis=0)
        downsampled_hist.append(sample_hist)

    return downsampled_hist

def process_videos(video_files):
    all_data = []

    for video_path in video_files:
        # Open the video file to get fps and frame count
        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        
        # Calculate start and end frames for the last 15 minutes, excluding the last 60 seconds
        duration_seconds = 15 * 60  # 15 minutes in seconds
        frames_to_process = duration_seconds * fps
        end_frame = total_frames - (60 * fps)  # Exclude the last 60 seconds
        start_frame = max(0, end_frame - frames_to_process)

        # Extract features
        optical_flow_df = extract_optical_flow(video_path, start_frame, end_frame)
        downsampled_hog = extract_hog_features(video_path, start_frame, end_frame)
        downsampled_hist = extract_color_histogram(video_path, start_frame, end_frame)

        # Ensure the downsampled data for HOG and color histogram are 1Hz and have 900 samples
        hog_df = pd.DataFrame(downsampled_hog, columns=[f'hog_{i}' for i in range(len(downsampled_hog[0]))])
        hist_df = pd.DataFrame(downsampled_hist, columns=[f'hist_{i}' for i in range(len(downsampled_hist[0]))])

        # Combine all features into a single DataFrame
        combined_df = optical_flow_df.copy()
        combined_df = pd.concat([combined_df, hog_df, hist_df], axis=1)
        combined_df['video_name'] = os.path.basename(video_path)  # Add video name for reference

        all_data.append(combined_df)

    # Concatenate all the individual DataFrames into one
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

# Example usage:
video_files = ['29_video.mkv', '14_video.mkv', '17_video.mkv', '23_video.mkv', '18_video.mkv']  # Add your video file paths here
final_dataframe = process_videos(video_files)

# Output the DataFrame
print(final_dataframe)

# Save to CSV if needed
final_dataframe.to_csv('video_features.csv', index=False)


NameError: name 'cv2' is not defined